In [1]:
import os
import sys
import numpy as np
import random
os.chdir('..')
os.getcwd()

'/home/demir/Desktop/physical_embedding'

In [2]:
from helper_classes import PL2VEC
from helper_classes import Parser
from helper_classes import DataAnalyser
import util as ut

In [3]:
## Set seet for number generator
random_state = 1
np.random.seed(random_state)
random.seed(random_state)

In [4]:
# DEFINE MODEL PARAMS
K = 5
num_of_dims = 50
bound_on_iter = 15
negative_constant = -1
e_release = 0.001
num_sample_from_clusters = 4
system_energy = 10

In [5]:
# Define paths
kg_root = 'KGs/DBpedia'
kg_path = kg_root + '/skos_categories_en.ttl.bz2'
dl_learner_path = '/home/demir/Desktop/physical_embedding/dllearner-1.3.0/bin/cli'

In [6]:
storage_path, experiment_folder = ut.create_experiment_folder()

In [7]:
parser = Parser(p_folder=storage_path)

model = PL2VEC(system_energy=system_energy)

analyser = DataAnalyser(p_folder=storage_path, execute_DL_Learner=dl_learner_path)

stats_corpus_info = parser.construct_comatrix(kg_path, bound=50000, bound_flag=True)
vocab_size = len(stats_corpus_info)


###### KG to PPMI Matrix  starts ######
KG to PPMI Matrix  took  1.4447062015533447  seconds



In [8]:
sys.getsizeof(stats_corpus_info) #returns bytes

1310816

In [9]:
P, N = parser.get_attactive_repulsive_entities(stats_corpus_info, K)

In [10]:
sys.getsizeof(P)+sys.getsizeof(N)

533952

In [11]:
# due to memory usage
del stats_corpus_info

In [12]:
ut.serializer(object_=N, path=parser.p_folder, serialized_name='Negative_URIs')
ut.serializer(object_=P, path=parser.p_folder, serialized_name='Positive_URIs')

In [13]:
# Efficient representation
holder = model.combine_information(P, N)
del P
del N

In [14]:
sys.getsizeof(holder)#returns bytes

253632

In [15]:
embeddings = model.randomly_initialize_embedding_space(vocab_size, num_of_dims)
# embeddings = model.initialize_with_svd(stats_corpus_info, num_of_dims)


In [16]:

learned_embeddings = model.start(e=embeddings,
                                 max_iteration=bound_on_iter, energy_release_at_epoch=e_release,
                                 holder=holder, negative_constant=negative_constant)
del embeddings
del holder



###### Generating Embeddings:  starts ######
EPOCH:  0
EPOCH:  1

 Epoch:  1
Previous norm 1247.6978800975821
New norm 1247.6978800976046
The differences in matrix norm  2.2509993868879974e-11
d(Semantically Similar)/d(Not Semantically Similar)  0.0037690731580803075
The state of equilibrium is reached.
Generating Embeddings:  took  22.45474648475647  seconds



In [17]:
representative_entities = analyser.pipeline_of_data_processing_single_run(learned_embeddings, num_sample_from_clusters)



###### Pipeline of DP  starts ######

###### Prune non resources  starts ######
Prune non resources  took  0.0028257369995117188  seconds


###### Pseudo labeling via DBSCAN  starts ######
Pseudo labeling via DBSCAN  took  1.1890599727630615  seconds


###### Sample from mean of clusters  starts ######
Sample from mean of clusters  took  0.7323789596557617  seconds

Pipeline of DP  took  1.9297902584075928  seconds



In [18]:

analyser.pipeline_of_single_evaluation_dl_learner(representative_entities)

# run DL learner
dl = analyser.generated_responds(experiment_folder)



###### DL-Learner  starts ######
Total Num of clusters  [ 0 -1]
DL-Learner  took  0.0046651363372802734  seconds

[('/home/demir/Desktop/physical_embedding/Experiments/2019-02-14 16:24:50.477852', ['0.conf', '-1.conf'])]
/home/demir/Desktop/physical_embedding/Experiments/2019-02-14 16:24:50.477852
['0.conf', '-1.conf']


In [19]:
dl

[['\n\n',
  '### 0.conf starts ###',
  'DL-Learner command line interface',
  "Initializing component 'ks' of type OWL File ...",
  "... initialized component 'ks' in 1ms. Status: OK",
  "Initializing component 'reasoner' of type closed world reasoner ...",
  'OntologyID(Anonymous-2)',
  'Loaded reasoner: Pellet (com.clarkparsia.pellet.owlapiv3.PelletReasoner)',
  'Materializing TBox...',
  '...TBox materialised in 189 ms.',
  "... initialized component 'reasoner' in 3s 482ms. Status: OK",
  "Initializing component 'op' of type rho refinement operator ...",
  "... initialized component 'op' in 4ms. Status: OK",
  "Initializing component 'fmeasure' of type FMeasure ...",
  "... initialized component 'fmeasure' in 0ms. Status: OK",
  "Initializing component 'lp' of type PosNegLPStandard ...",
  "... initialized component 'lp' in 0ms. Status: OK",
  "Initializing component 'alg' of type CELOE ...",
  "... initialized component 'alg' in 1ms. Status: OK",
  'Running algorithm instance "alg"